In [1]:
import pandas as pd
import math
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
import healpy as hp
import matplotlib.pyplot as plt
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky, wcsutils
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from tqdm import tqdm
import random
import time
import multiprocessing as mp
import numpy as np
import random
import time
import multiprocessing as mp
import numpy as np

---

# Read in data

#### Catalog

In [2]:
ard_not_adv = pd.read_csv('../data/ard_not_adv.csv', skiprows = 0)

In [3]:
# ras_good = []
# decs_good = []

In [4]:
# ras_fake = ard_not_adv['amf_ra']+random.uniform(-1,1)
# decs_fake = ard_not_adv['amf_dec']+random.uniform(-1,1)

# for i in tqdm(range(len(ras_fake))):
#     matches = ard_not_adv.loc[(ard_not_adv['amf_ra'].sub(ras_fake[i]).abs()<1) & (ard_not_adv['amf_dec'].sub(decs_fake[i]).abs()<1)]
#     if len(matches)==0:  
#         ras_good.append(ras_fake[i])
#         decs_good.append(decs_fake[i])

In [5]:
# plt.scatter(ard_not_adv['amf_ra'],ard_not_adv['amf_dec'],s=0.1)
# plt.scatter(ras_good,decs_good,s=0.1,color='red')
# # plt.xlim(100,250)

In [6]:
# len(ras_good)

---

#### Planck

In [7]:
# pl = hp.read_map("../data/HFI_SkyMap_217-field-IQU_2048_R3.00_full.fits")
# pl[pl<-1e23]=0

# shape,wcs = enmap.fullsky_geometry(res=5.0 * utils.arcmin,proj='car')
# map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)

#### ROSAT

In [8]:
# ros = hp.read_map("../data/rass_healpix_countsec.fits")
# mask = hp.read_map("../data/mask_comprass.fits")

# ros[ros<-1e23]=0

# masked_ros = []
# for i in tqdm(range(len(mask))):
#     masked_ros.append(0)
#     if mask[i]:
#         masked_ros[i]=ros[i]

# masked_ros = np.array(masked_ros)

# shape,wcs = enmap.fullsky_geometry(res=5.0 * utils.arcmin,proj='car')
# map_pix = reproject.enmap_from_healpix(masked_ros,shape=shape,wcs=wcs)

#### ACT

In [9]:
map_pix = enmap.read_map("../data/act_dr5.01_s08s18_AA_f090_daynight_map.fits")

---

# Intensity extraction

In [10]:
# Full
# ras = np.linspace(0,360,142)
# decs1 = np.linspace(-90,-25,71)
# decs2 = np.linspace(25,90,71)
# decs = np.concatenate((decs1,decs2))

# Fake
# ras = np.loadtxt('fakeras.csv',delimiter=', ')
# decs = np.loadtxt('fakedecs.csv',delimiter=', ')

# Real
ras  = ard_not_adv['amf_ra']
decs = ard_not_adv['amf_dec']

N = len(ras)
stack_CMB_kSZ           = 0
stack_CMB_kSZ_deproject = 0
c = 0

In [11]:
signal = []

# Full map
# for j in tqdm(range(N)):
#     for i in range(N):
#         stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[j], ras[i]]),r=3*utils.arcmin)

#         if stamp is None:
#             signal.append(0)
#             continue
#         elif stamp[0][0][0]==0.0:
#             signal.append(0)
#             continue

#         signal.append(sum(sum(stamp[0]))/9)
#         ras_temp.append(ras[i])
#         decs_temp.append(decs[j])

# Fake
for i in tqdm(range(N)):
    stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=3*utils.arcmin)
    if stamp is None: signal.append(-9999)
    elif stamp[0][0][0]==0.0: signal.append(-9999)
    else: signal.append(sum(sum(stamp[0]))/(25*25))
#     plt.imshow(stamp[0])
#     plt.show()

# Real
# for i in tqdm(range(N)):
#     stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs[i], ras[i]]),r=3*utils.arcmin)

#     if stamp is None:
#         signal.append(0)
#         continue
#     elif stamp[0][0][0]==0.0:
#         signal.append(0)
#         continue

#     signal.append(sum(sum(stamp[0]))/9)

100%|██████████| 21006/21006 [04:46<00:00, 73.42it/s]


In [12]:
len(signal)

21006

In [13]:
np.savetxt("act_intensity090.csv",
           signal,
           delimiter=", ",
           fmt='% s')

# np.savetxt("fakeras.csv",
#            ras_good,
#            delimiter=", ",
#            fmt='% s')

# np.savetxt("fakedecs.csv",
#            decs_good,
#            delimiter=", ",
#            fmt='% s')